# Lab 14b: Publish Foundry Agents

This lab demonstrates how to **publish a Foundry agent** as an Agent Application with a dedicated endpoint, identity, and governance capabilities.

## What You'll Learn

| Step | Description |
|------|-------------|
| **1. Deploy Infrastructure** | Create Foundry account, project, and model deployment |
| **2. Create Agent** | Build a simple Space Facts agent |
| **3. Publish Application** | Create Agent Application via ARM API |
| **4. Create Deployment** | Deploy agent version to the application |
| **5. Test Endpoint** | Call published agent via Responses API |
| **6. Teams Integration** | (Optional) Publish to Teams via Portal |

## Publishing Benefits

| Without Publishing | With Publishing |
|-------------------|----------------|
| Shared project identity | Dedicated agent identity |
| Project-level access required | Shareable without project access |
| No external distribution | Stable endpoint for consumers |
| Shared user data | Isolated user data per caller |

## Prerequisites

- ✅ Azure subscription with Cognitive Services access
- ✅ **Azure AI Project Manager** role (to publish agents)
- ✅ **Azure AI User** role (to chat with published agent)

## References

- 📚 [Publish and share agents in Microsoft Foundry](https://learn.microsoft.com/azure/ai-services/agents/how-to/publish-agents)
- 📚 [Agent identity concepts](https://learn.microsoft.com/azure/ai-services/agents/concepts/agent-identity)

## Step 1: Install Dependencies

In [130]:
!pip install azure-ai-projects==2.0.0b2 azure-identity requests -q

## Step 2: Configure Environment

Set up resource configuration and get Azure credentials.

In [131]:
import os
import subprocess
import json
import requests
import random
import time
from IPython.display import display, Markdown, HTML, clear_output

# Resource configuration
RG = "foundry-m365-lab"
LOCATION = "swedencentral"  # Change to your preferred region
MODEL_NAME = "gpt-4.1-mini"
AGENT_NAME = "SpaceFunFactsAgent"

# Get current user info
PRINCIPAL_ID = subprocess.run(
    'az ad signed-in-user show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

SUBSCRIPTION_ID = subprocess.run(
    'az account show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

TENANT_ID = subprocess.run(
    'az account show --query tenantId -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

display(Markdown(f'''
### ✅ Configuration Ready

| Setting | Value |
|---------|-------|
| Resource Group | `{RG}` |
| Location | `{LOCATION}` |
| Model | `{MODEL_NAME}` |
| Agent Name | `{AGENT_NAME}` |
| Subscription | `<your-subscription-id>` |
| Principal ID | `<your-principal-id>` |
'''))


### ✅ Configuration Ready

| Setting | Value |
|---------|-------|
| Resource Group | `foundry-m365-lab` |
| Location | `swedencentral` |
| Model | `gpt-4.1-mini` |
| Agent Name | `SpaceFunFactsAgent` |
| Subscription | `<your-subscription-id>` |
| Principal ID | `<your-principal-id>` |


## Step 3: Deploy Infrastructure

Deploy Foundry account, project, and model deployment. ⏱️ Takes ~5 minutes.

In [ ]:
!az group create -n "{RG}" -l "{LOCATION}" -o table

In [133]:
!az deployment group create -g "{RG}" --template-file spoke.bicep \
    -p deployerPrincipalId="{PRINCIPAL_ID}" \
    -p modelName="{MODEL_NAME}" \
    -o table

Name    State      Timestamp                         Mode         ResourceGroup
------  ---------  --------------------------------  -----------  ----------------
spoke   Succeeded  2026-01-20T15:22:44.341455+00:00  Incremental  foundry-m365-lab


In [ ]:
# Get deployment outputs
outputs = json.loads(subprocess.run(
    f'az deployment group show -g "{RG}" -n spoke --query properties.outputs -o json',
    shell=True, capture_output=True, text=True
).stdout)

ACCOUNT_NAME = outputs['accountName']['value']
PROJECT_NAME = outputs['projectName']['value']
PROJECT_ENDPOINT = outputs['projectEndpoint']['value']
MODEL_DEPLOYMENT = outputs['modelDeploymentName']['value']

display(Markdown(f'''
### ✅ Infrastructure Deployed!

| Resource | Value |
|----------|-------|
| AI Account | `{ACCOUNT_NAME}` |
| Project | `{PROJECT_NAME}` |
| Model Deployment | `{MODEL_DEPLOYMENT}` |
| Endpoint | `https://<account>.services.ai.azure.com/...` |
'''))

## Step 4: Wait for RBAC Propagation

Azure role assignments can take a minute to propagate.

In [135]:
for i in range(60, 0, -10):
    clear_output(wait=True)
    print(f"⏳ Waiting for RBAC to propagate... {i}s")
    time.sleep(10)

clear_output(wait=True)
print("✅ RBAC permissions ready!")

✅ RBAC permissions ready!


## Step 5: Create Space Facts Agent 🚀

Create a simple agent that shares interesting space facts.

In [136]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition

credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

SPACE_FACTS_INSTRUCTIONS = """
You are a friendly Space Facts expert! Your job is to share fascinating facts about space, planets, stars, galaxies, and space exploration.

Guidelines:
- Be enthusiastic and engaging
- Share accurate, interesting space facts
- Include fun comparisons to help users understand scale
- If asked about non-space topics, politely redirect to space facts
- Use emojis to make responses more fun 🚀🌟🪐

Example facts you might share:
- The Great Red Spot on Jupiter is a storm that has been raging for over 400 years
- Olympus Mons on Mars is the largest volcano in the solar system (3x the height of Everest!)
- A day on Venus is longer than its year
- Neutron stars can spin at 600+ rotations per second
"""

# Create the agent using the dedicated model deployment
agent = project_client.agents.create_version(
    agent_name=AGENT_NAME,
    definition=PromptAgentDefinition(
        model=MODEL_DEPLOYMENT,
        instructions=SPACE_FACTS_INSTRUCTIONS
    )
)

AGENT_VERSION = agent.version

display(Markdown(f'''
### ✅ Agent Created!

| Property | Value |
|----------|-------|
| Name | `{agent.name}` |
| Version | `{AGENT_VERSION}` |
| Model | `{MODEL_DEPLOYMENT}` |
'''))


### ✅ Agent Created!

| Property | Value |
|----------|-------|
| Name | `SpaceFunFactsAgent` |
| Version | `1` |
| Model | `gpt-4.1-mini` |


## Step 6: Test the Agent Locally

Before publishing, let's verify the agent works correctly.

In [137]:
openai_client = project_client.get_openai_client()

def ask_agent(question: str) -> str:
    """Ask the space facts agent a question."""
    response = openai_client.responses.create(
        input=question,
        extra_body={
            "agent": {
                "name": AGENT_NAME,
                "version": AGENT_VERSION,
                "type": "agent_reference"
            }
        }
    )
    return response.output_text

# Test the agent
answer = ask_agent("Tell me a fun fact about Mars!")
display(Markdown(f'''
### 🚀 Agent Response

**Question:** Tell me a fun fact about Mars!

**Answer:** {answer}
'''))


### 🚀 Agent Response

**Question:** Tell me a fun fact about Mars!

**Answer:** Absolutely! 🚀 Did you know that Mars has the tallest volcano in the entire solar system? It's called Olympus Mons, and it stands about 13.6 miles (22 kilometers) high—roughly three times the height of Mount Everest! Imagine a mountain so huge that if you were standing at its base, the summit would be way above the cruising altitude of a commercial airplane. How cool is that? 🪐🌋


---

## Step 7: Publish Agent Application (ARM REST API)

Publishing creates an **Agent Application** resource with:
- Dedicated invocation URL (`/applications/{appName}/protocols/openai`)
- Independent **Agent Identity** (separate from project identity)
- RBAC-enabled endpoint for external consumers

We use the official **Azure Resource Manager (ARM) REST API** to create the application and deployment.

In [138]:
# Create Agent Application using ARM REST API
# Reference: https://learn.microsoft.com/azure/ai-services/agents/how-to/publish-agents

API_VERSION = "2025-10-01-preview"  # Use the supported API version
APPLICATION_NAME = AGENT_NAME  # Use agent name as application name

# ARM API endpoint for creating the application
APP_ARM_URL = (
    f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}"
    f"/resourceGroups/{RG}/providers/Microsoft.CognitiveServices"
    f"/accounts/{ACCOUNT_NAME}/projects/{PROJECT_NAME}"
    f"/applications/{APPLICATION_NAME}?api-version={API_VERSION}"
)

app_payload = {
    "properties": {
        "displayName": AGENT_NAME,
        "agents": [{"agentName": AGENT_NAME}]
    }
}

print(f"🚀 Creating Agent Application: {APPLICATION_NAME}")
print(f"   Using ARM API (version {API_VERSION})...")

result = subprocess.run(
    f'az rest --method PUT --uri "{APP_ARM_URL}" --body \'{json.dumps(app_payload)}\'',
    shell=True, capture_output=True, text=True
)

if result.returncode == 0:
    app_result = json.loads(result.stdout)
    app_props = app_result.get('properties', {})
    
    # Extract the agent identity details
    identity = app_props.get('defaultInstanceIdentity', {})
    APP_CLIENT_ID = identity.get('clientId', '')
    APP_PRINCIPAL_ID = identity.get('principalId', '')
    
    # Construct the application base URL
    BASE_URL = app_props.get('baseUrl', f"https://{ACCOUNT_NAME}.services.ai.azure.com/api/projects/{PROJECT_NAME}/applications/{APPLICATION_NAME}")
    
    display(Markdown(f'''
### ✅ Agent Application Created!

| Property | Value |
|----------|-------|
| Application Name | `{APPLICATION_NAME}` |
| Display Name | `{app_props.get('displayName', 'N/A')}` |
| Base URL | `https://<account>.services.ai.azure.com/...` |
| Client ID | `<agent-client-id>` |
| Principal ID | `<agent-principal-id>` |
| Provisioning State | `{app_props.get('provisioningState', 'N/A')}` |
'''))
else:
    print(f"❌ Failed to create application")
    print(result.stderr or result.stdout)

🚀 Creating Agent Application: SpaceFunFactsAgent
   Using ARM API (version 2025-10-01-preview)...



### ✅ Agent Application Created!

| Property | Value |
|----------|-------|
| Application Name | `SpaceFunFactsAgent` |
| Display Name | `SpaceFunFactsAgent` |
| Base URL | `https://<account>.services.ai.azure.com/...` |
| Client ID | `<agent-client-id>` |
| Principal ID | `<agent-principal-id>` |
| Provisioning State | `Succeeded` |


In [139]:
# Create Deployment for the Agent Application
# This creates the running instance that routes traffic to the agent version

DEPLOYMENT_NAME = "default"

DEPLOY_ARM_URL = (
    f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}"
    f"/resourceGroups/{RG}/providers/Microsoft.CognitiveServices"
    f"/accounts/{ACCOUNT_NAME}/projects/{PROJECT_NAME}"
    f"/applications/{APPLICATION_NAME}/agentdeployments/{DEPLOYMENT_NAME}?api-version={API_VERSION}"
)

deploy_payload = {
    "properties": {
        "displayName": f"{AGENT_NAME} Deployment",
        "deploymentType": "Managed",
        "protocols": [
            {"protocol": "responses", "version": "1.0"}
        ],
        "agents": [
            {"agentName": AGENT_NAME, "agentVersion": AGENT_VERSION}
        ]
    }
}

print(f"🚀 Creating Deployment: {DEPLOYMENT_NAME}")

result = subprocess.run(
    f'az rest --method PUT --uri "{DEPLOY_ARM_URL}" --body \'{json.dumps(deploy_payload)}\'',
    shell=True, capture_output=True, text=True
)

if result.returncode == 0:
    deploy_result = json.loads(result.stdout)
    deploy_props = deploy_result.get('properties', {})
    
    # Construct endpoint URLs
    RESPONSES_URL = f"{BASE_URL}/protocols/openai"
    ACTIVITY_URL = f"{BASE_URL}/protocols/activityprotocol"
    
    display(Markdown(f'''
### ✅ Deployment Created!

| Property | Value |
|----------|-------|
| Deployment Name | `{DEPLOYMENT_NAME}` |
| Agent | `{AGENT_NAME}` v`{AGENT_VERSION}` |
| Deployment Type | `{deploy_props.get('deploymentType', 'Managed')}` |
| State | `{deploy_props.get('state', 'N/A')}` |
| Protocols | responses, activityprotocol |

**Endpoints:**
- **Responses API**: `https://<account>.services.ai.azure.com/.../protocols/openai`
- **Activity Protocol**: `https://<account>.services.ai.azure.com/.../protocols/activityprotocol`
'''))
else:
    print(f"❌ Failed to create deployment")
    print(result.stderr or result.stdout)

🚀 Creating Deployment: default



### ✅ Deployment Created!

| Property | Value |
|----------|-------|
| Deployment Name | `default` |
| Agent | `SpaceFunFactsAgent` v`1` |
| Deployment Type | `Managed` |
| State | `Running` |
| Protocols | responses, activityprotocol |

**Endpoints:**
- **Responses API**: `https://<account>.services.ai.azure.com/.../protocols/openai`
- **Activity Protocol**: `https://<account>.services.ai.azure.com/.../protocols/activityprotocol`


## Step 8: Assign RBAC to Agent Identity

The published agent has its own **Agent Identity**. Grant it **Azure AI User** role on the project so it can execute agent operations when invoked.

> ⚠️ **Critical**: Without this step, the agent will fail with an authorization error when called via the application endpoint.

In [ ]:
# Azure AI User role ID (official from Microsoft Learn docs)
AI_USER_ROLE = "53ca6127-db72-4b80-b1b0-d745d6d5456d"

# First, get the Service Principal object ID from the Client ID
# The ARM API returns the Application object ID, but we need the Service Principal
print(f"🔍 Looking up Service Principal for Client ID: {APP_CLIENT_ID}")

sp_result = subprocess.run(
    f'az ad sp show --id "{APP_CLIENT_ID}" --query id -o tsv',
    shell=True, capture_output=True, text=True
)

if sp_result.returncode == 0 and sp_result.stdout.strip():
    SP_OBJECT_ID = sp_result.stdout.strip()
    print(f"   Found Service Principal: {SP_OBJECT_ID}")
else:
    # Fallback: use the principal ID from ARM response
    SP_OBJECT_ID = APP_PRINCIPAL_ID
    print(f"   Using ARM Principal ID: {SP_OBJECT_ID}")

# Project scope
PROJECT_SCOPE = f"/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RG}/providers/Microsoft.CognitiveServices/accounts/{ACCOUNT_NAME}/projects/{PROJECT_NAME}"

print(f"\n🔐 Assigning Azure AI User role to agent identity...")

result = subprocess.run(
    f'az role assignment create --assignee-object-id "{SP_OBJECT_ID}" '
    f'--assignee-principal-type ServicePrincipal '
    f'--role "{AI_USER_ROLE}" '
    f'--scope "{PROJECT_SCOPE}" '
    f'-o json',
    shell=True, capture_output=True, text=True
)

if result.returncode == 0 or "already exists" in result.stderr.lower():
    print("✅ Azure AI User role assigned to agent identity!")
else:
    print(f"⚠️ Role assignment result: {result.stderr or result.stdout}")

## Step 9: Test Published Agent

Now let's test the published agent through both available protocols.

In [141]:
# Test the Activity Protocol endpoint (used by Teams/Bot Service)
import uuid
from azure.identity import DefaultAzureCredential

token = DefaultAzureCredential().get_token("https://ai.azure.com/.default").token
ACTIVITY_ENDPOINT = f"{BASE_URL}/protocols/activityprotocol?api-version=2025-11-15-preview"

activity = {
    "type": "message",
    "id": str(uuid.uuid4()),
    "timestamp": "2026-01-20T12:00:00Z",
    "channelId": "test",
    "from": {"id": "user-1", "name": "Test User"},
    "conversation": {"id": str(uuid.uuid4())},
    "recipient": {"id": "bot", "name": AGENT_NAME},
    "text": "Tell me a fascinating fact about black holes!",
    "serviceUrl": "https://example.com/callback"  # Webhook URL for async response
}

print("🚀 Testing Activity Protocol endpoint...")
response = requests.post(ACTIVITY_ENDPOINT, json=activity, 
                          headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"})

if response.status_code == 202:
    display(Markdown(f'''
### ✅ Activity Protocol Working!

The endpoint returned **202 Accepted** - the agent application is correctly configured.

| Property | Value |
|----------|-------|
| Status | `202 Accepted` |
| Protocol | Activity Protocol (Bot Framework) |

> **Note:** The Activity Protocol is **asynchronous**. In production, Azure Bot Service or Teams 
> handles the callback automatically. The response would be delivered to the `serviceUrl` webhook.
> For synchronous testing, use the Responses API below.
'''))
elif response.status_code in [401, 403]:
    print(f"❌ Authorization error ({response.status_code}) - wait for RBAC to propagate")
else:
    print(f"❌ Error: {response.status_code} - {response.text[:300]}")

🚀 Testing Activity Protocol endpoint...



### ✅ Activity Protocol Working!

The endpoint returned **202 Accepted** - the agent application is correctly configured.

| Property | Value |
|----------|-------|
| Status | `202 Accepted` |
| Protocol | Activity Protocol (Bot Framework) |

> **Note:** The Activity Protocol is **asynchronous**. In production, Azure Bot Service or Teams 
> handles the callback automatically. The response would be delivered to the `serviceUrl` webhook.
> For synchronous testing, use the Responses API below.


### Responses API (Synchronous)

For development and testing, the **Responses API** provides a synchronous, OpenAI-compatible interface:

| Protocol | Use Case | Response |
|----------|----------|----------|
| **Activity Protocol** | Teams, Bot Service, Copilot Studio | Async (202 + webhook callback) |
| **Responses API** | Development, testing, direct integrations | Synchronous |

In [ ]:
# Alternative: Use the Responses API for synchronous testing
# This is easier for local development since it doesn't require a public webhook
from azure.identity import DefaultAzureCredential

token = DefaultAzureCredential().get_token("https://ai.azure.com/.default").token

# Responses API endpoint (OpenAI-compatible)
RESPONSES_ENDPOINT = f"{BASE_URL}/protocols/openai/responses?api-version=2025-11-15-preview"

# Note: Don't pass "model" when using the application endpoint - the agent IS the model
payload = {
    "input": "Tell me a fascinating fact about black holes!"
}

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

print("🚀 Testing published agent via Responses API (synchronous)...")
print(f"   Endpoint: {RESPONSES_ENDPOINT}")

response = requests.post(RESPONSES_ENDPOINT, json=payload, headers=headers)
print(f"   Status: {response.status_code}")

if response.status_code == 200:
    result = response.json()
    
    # Extract the text response
    output = result.get("output", [])
    text_response = ""
    for item in output:
        if item.get("type") == "message":
            for content in item.get("content", []):
                if content.get("type") == "output_text":
                    text_response = content.get("text", "")
    
    display(Markdown(f'''
### ✅ Agent Response

**Question:** Tell me a fascinating fact about black holes!

**Answer:** {text_response}

---
*Protocol:* Responses API (OpenAI-compatible)  
*Response ID:* {result.get("id", "N/A")}
'''))
    
elif response.status_code == 401 or response.status_code == 403:
    print(f"❌ Authorization error ({response.status_code})")
    print("\n⚠️ Wait 1-2 minutes for RBAC to propagate and try again.")
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text[:1000] if response.text else "")

## Step 10: Publish to M365 

To enable Microsoft Teams integration, use the **Microsoft AI Foundry Portal**. The portal handles:
- Creating the Azure Bot Service with Teams channel
- Configuring the Teams app manifest
- Publishing to your organization's Teams app catalog

> 💡 Teams integration requires interactive browser authentication with a compliant device.

In [ ]:
# Serve an interactive guide page and open it in the browser
import base64
import uuid
import threading
import socket
import webbrowser
from http.server import HTTPServer, SimpleHTTPRequestHandler
from pathlib import Path

# Build portal URL
sub_bytes = uuid.UUID(SUBSCRIPTION_ID).bytes
encoded_sub = base64.urlsafe_b64encode(sub_bytes).decode('utf-8').rstrip('=')
PORTAL_DIRECT_URL = (
    f"https://ai.azure.com/nextgen/r/{encoded_sub},{RG},,{ACCOUNT_NAME},{PROJECT_NAME}"
    f"/build/agents/{AGENT_NAME}/build?version={AGENT_VERSION}"
)

# Load and customize the HTML template
html_path = Path("publish_guide.html")
html_content = html_path.read_text()
html_content = html_content.replace("{{PORTAL_URL}}", PORTAL_DIRECT_URL)
html_content = html_content.replace("{{AGENT_NAME}}", AGENT_NAME)
html_content = html_content.replace("{{AGENT_VERSION}}", str(AGENT_VERSION))
html_content = html_content.replace("{{APPLICATION_NAME}}", APPLICATION_NAME)
html_content = html_content.replace("{{PROJECT_NAME}}", PROJECT_NAME)

# Write customized HTML
customized_path = Path("_publish_guide_temp.html")
customized_path.write_text(html_content)

# Simple HTTP handler that serves from current directory
class GuideHandler(SimpleHTTPRequestHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=str(Path.cwd()), **kwargs)
    
    def log_message(self, format, *args):
        pass  # Suppress logging

# Find an available port
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]

GUIDE_PORT = find_free_port()

# Start server in background
server = HTTPServer(('127.0.0.1', GUIDE_PORT), GuideHandler)
server_thread = threading.Thread(target=server.serve_forever, daemon=True)
server_thread.start()

# Open browser to the guide page
guide_url = f"http://localhost:{GUIDE_PORT}/_publish_guide_temp.html"
webbrowser.open(guide_url)

display(Markdown(f'''
## 📱 Publish to Microsoft Teams

A guide page has been opened in your browser with step-by-step instructions.

---

### Quick Summary:

1. **In the Foundry Portal:** Click "Publish" → "Publish to Teams" → Fill metadata → Publish
2. **In Microsoft Teams:** Open Copilot → Click @ → Search for "{AGENT_NAME}"

### Direct Links:

| Resource | Link |
|----------|------|
| 🚀 Foundry Portal | [Open Agent in Portal](https://ai.azure.com/...) |
| 📚 Documentation | [Publish agents docs](https://learn.microsoft.com/azure/ai-services/agents/how-to/publish-agents) |
'''))

print("🌐 Guide opened in browser")

---

## 🎉 Summary

Your agent is now published as an **Agent Application** with its own identity and stable endpoint!

In [144]:
# Summary of what was created
display(Markdown(f'''
## 🎉 Publication Complete!

### 📊 Resources Created

| Component | Details |
|-----------|---------|  
| **Agent** | `{AGENT_NAME}` version `{AGENT_VERSION}` |
| **Application** | `{APPLICATION_NAME}` |
| **Deployment** | `{DEPLOYMENT_NAME}` (Managed) |
| **Identity** | `<agent-client-id>` |

---

### 🔗 Endpoints

| Protocol | URL |
|----------|-----|
| **Responses API** | `https://<account>.services.ai.azure.com/.../protocols/openai` |
| **Activity Protocol** | `https://<account>.services.ai.azure.com/.../protocols/activityprotocol` |

---

### 📖 How to Consume

**Python (OpenAI SDK):**
```python
from openai import OpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

client = OpenAI(
    api_key=get_bearer_token_provider(DefaultAzureCredential(), "https://ai.azure.com/.default")(),
    base_url="<your-responses-url>",
    default_query={{"api-version": "2025-11-15-preview"}}
)

response = client.responses.create(input="Tell me a space fact!")
print(response.output_text)
```

---

### 🔐 Access Control

To grant users access to call your published agent:

```bash
az role assignment create \\
    --assignee "<user-or-group-object-id>" \\
    --role "Azure AI User" \\
    --scope "/subscriptions/<sub>/resourceGroups/<rg>/providers/Microsoft.CognitiveServices/accounts/<account>/projects/<project>/applications/<app>"
```
'''))


## 🎉 Publication Complete!

### 📊 Resources Created

| Component | Details |
|-----------|---------|  
| **Agent** | `SpaceFunFactsAgent` version `1` |
| **Application** | `SpaceFunFactsAgent` |
| **Deployment** | `default` (Managed) |
| **Identity** | `<agent-client-id>` |

---

### 🔗 Endpoints

| Protocol | URL |
|----------|-----|
| **Responses API** | `https://<account>.services.ai.azure.com/.../protocols/openai` |
| **Activity Protocol** | `https://<account>.services.ai.azure.com/.../protocols/activityprotocol` |

---

### 📖 How to Consume

**Python (OpenAI SDK):**
```python
from openai import OpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

client = OpenAI(
    api_key=get_bearer_token_provider(DefaultAzureCredential(), "https://ai.azure.com/.default")(),
    base_url="<your-responses-url>",
    default_query={"api-version": "2025-11-15-preview"}
)

response = client.responses.create(input="Tell me a space fact!")
print(response.output_text)
```

---

### 🔐 Access Control

To grant users access to call your published agent:

```bash
az role assignment create \
    --assignee "<user-or-group-object-id>" \
    --role "Azure AI User" \
    --scope "/subscriptions/<sub>/resourceGroups/<rg>/providers/Microsoft.CognitiveServices/accounts/<account>/projects/<project>/applications/<app>"
```


### Agent in M365

![Agent communicating in Teams](preview.png)

---

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Agent Application** | ARM resource with stable endpoint and dedicated identity |
| **Deployment** | Running instance routing traffic to specific agent version |
| **Agent Identity** | Independent service principal for RBAC and audit |
| **Responses Protocol** | OpenAI-compatible API for agent invocation |
| **Activity Protocol** | Bot Framework protocol for Teams/channels |

## Cleanup (Optional)

Uncomment and run to delete all resources created in this lab.

In [145]:
# Uncomment to delete resources
# !az group delete -n "{RG}" --yes --no-wait
# print("🗑️ Cleanup initiated - resources will be deleted in the background")

---

## Additional Resources

### Official Documentation

- 📚 [Publish and share agents in Microsoft Foundry](https://learn.microsoft.com/azure/ai-services/agents/how-to/publish-agents)
- 📚 [Agent identity concepts in Foundry](https://learn.microsoft.com/azure/ai-services/agents/concepts/agent-identity)
- 📚 [Azure Bot Service Documentation](https://learn.microsoft.com/azure/bot-service/)

### API Reference

| API | Description |
|-----|-------------|
| `PUT .../applications/{name}` | Create/update Agent Application |
| `PUT .../agentdeployments/{name}` | Create/update deployment |
| `POST /protocols/openai/responses` | Invoke agent (Responses API) |
| `POST /protocols/activityprotocol` | Bot Framework activity protocol |